In [2]:
# Summary
print(gwas_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SNP         236 non-null    object
 1   CHROMOSOME  236 non-null    int64 
 2   POSITION    236 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.7+ KB
None


In [3]:
print(gwas_df.columns[:10])

Index(['SNP', 'CHROMOSOME', 'POSITION'], dtype='object')


In [4]:
diagram_df = pd.read_csv(r"C:\Users\gargi\Desktop\Publication\Transcriptomic_and_Epigenomic_Features\DIAGRAMv3.2012DEC17.csv")

diagram_df.head(5)

,SNP,CHROMOSOME,POSITION,RISK_ALLELE,OTHER_ALLELE,P_VALUE,OR,OR_95L,OR_95U,N_CASES,N_CONTROLS
0,rs12565286,1,711153,G,C,0.16,1.11,0.96,1.30,4040,17875
1,rs3094315,1,742429,A,G,0.82,1.01,0.95,1.07,6673,25149
2,rs2905035,1,765522,G,A,0.36,1.04,0.96,1.11,4984,23960
3,rs2980319,1,766985,T,A,0.39,1.03,0.97,1.09,7575,27012
4,rs4040617,1,769185,A,G,0.50,1.03,0.95,1.11,4984,23960


In [5]:
# Summary
print(diagram_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2473441 entries, 0 to 2473440
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   SNP           object 
 1   CHROMOSOME    int64  
 2   POSITION      int64  
 3   RISK_ALLELE   object 
 4   OTHER_ALLELE  object 
 5   P_VALUE       float64
 6   OR            float64
 7   OR_95L        float64
 8   OR_95U        float64
 9   N_CASES       int64  
 10  N_CONTROLS    int64  
dtypes: float64(4), int64(4), object(3)
memory usage: 207.6+ MB
None


In [6]:
print(diagram_df.columns[:10])

Index(['SNP', 'CHROMOSOME', 'POSITION', 'RISK_ALLELE', 'OTHER_ALLELE',
       'P_VALUE', 'OR', 'OR_95L', 'OR_95U', 'N_CASES'],
      dtype='object')


In [7]:
annotation_df = pd.read_csv(r"C:\Users\gargi\Desktop\Publication\Transcriptomic_and_Epigenomic_Features\bwMtrPidtktV3hgO.csv")  

annotation_df.head()

,rsID,Consequence,IMPACT,Gene,SIFT,PolyPhen,PHENOTYPES
0,#Uploaded_variation,Consequence,IMPACT,SYMBOL,DOMAINS,miRNA,ALFA_Latin_American_2_AN
1,rs1801282,missense_variant,MODERATE,PPARG,AFDB-ENSP_mappings:AF-P37231-F1,-,850
2,rs1801282,missense_variant,MODERATE,PPARG,AFDB-ENSP_mappings:AF-P37231-F1,-,850
3,rs1801282,"missense_variant,NMD_transcript_variant",MODERATE,PPARG,AFDB-ENSP_mappings:AF-E9PFV3-F1,-,850
4,rs1801282,"missense_variant,NMD_transcript_variant",MODERATE,PPARG,AFDB-ENSP_mappings:AF-E9PFV3-F1,-,850


In [8]:
# Summary
print(annotation_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   rsID         293 non-null    object
 1   Consequence  293 non-null    object
 2   IMPACT       293 non-null    object
 3   Gene         293 non-null    object
 4   SIFT         293 non-null    object
 5   PolyPhen     293 non-null    object
 6   PHENOTYPES   293 non-null    object
dtypes: object(7)
memory usage: 16.1+ KB
None


In [9]:
print(annotation_df.columns[:10])

Index(['rsID', 'Consequence', 'IMPACT', 'Gene', 'SIFT', 'PolyPhen',
       'PHENOTYPES'],
      dtype='object')


## Standardize column names

In [10]:
gwas_df.rename(columns={"SNP": "rsID"}, inplace=True)
diagram_df.rename(columns={"SNP": "rsID"}, inplace=True)
# annotation_df already has 'rsID' — no need to rename

## Merge GWAS SNPs with DIAGRAM GWAS data

In [11]:
merged_1 = pd.merge(gwas_df, diagram_df, on="rsID", how="inner")

## Merge with functional annotations

In [12]:
merged_final = pd.merge(merged_1, annotation_df, on="rsID", how="inner")


## Drop duplicates

In [13]:
merged_final.drop_duplicates(inplace=True)

## Save and inspect

In [15]:
merged_final.to_csv(r"C:\Users\gargi\Desktop\Publication\Transcriptomic_and_Epigenomic_Features\cleaned_merged_gwas_diagram_annotations.csv", index=False)
print(f"Merged data shape: {merged_final.shape}")
print(merged_final[['rsID', 'Consequence', 'IMPACT', 'P_VALUE']].head())

Merged data shape: (255, 19)
         rsID                              Consequence    IMPACT       P_VALUE
0   rs6771792                                        -         -  3.100000e-08
1  rs11709077                                        -         -  1.100000e-09
2   rs1801282                         missense_variant  MODERATE  5.000000e-09
4   rs1801282  missense_variant,NMD_transcript_variant  MODERATE  5.000000e-09
6   rs1801282                         missense_variant  MODERATE  5.000000e-09


## Filter High-Impact Functional Variants

In [16]:
coding_terms = ['missense_variant', 'stop_gained', 'splice_acceptor_variant', 'splice_donor_variant']
filtered_df = merged_final[merged_final['Consequence'].str.contains('|'.join(coding_terms), na=False)]
print(filtered_df[['rsID', 'Consequence', 'IMPACT', 'P_VALUE']])


           rsID                              Consequence    IMPACT  \
2     rs1801282                         missense_variant  MODERATE   
4     rs1801282  missense_variant,NMD_transcript_variant  MODERATE   
6     rs1801282                         missense_variant  MODERATE   
8     rs1801282                         missense_variant  MODERATE   
33    rs1801214                         missense_variant  MODERATE   
36    rs1801214                         missense_variant  MODERATE   
39    rs1801214                         missense_variant  MODERATE   
42    rs1801214                         missense_variant  MODERATE   
44    rs1801214                         missense_variant  MODERATE   
45    rs1801214                         missense_variant  MODERATE   
54    rs1801214                         missense_variant  MODERATE   
134  rs13266634                         missense_variant  MODERATE   
136  rs13266634                         missense_variant  MODERATE   
142  rs13266634     

## Sort by GWAS Significance

In [17]:
top_snps = filtered_df.sort_values(by="P_VALUE", ascending=True).head(10)
print(top_snps[['rsID', 'Gene', 'Consequence', 'P_VALUE']])


           rsID     Gene                              Consequence  \
134  rs13266634  SLC30A8                         missense_variant   
136  rs13266634  SLC30A8                         missense_variant   
142  rs13266634  SLC30A8                         missense_variant   
2     rs1801282    PPARG                         missense_variant   
4     rs1801282    PPARG  missense_variant,NMD_transcript_variant   
6     rs1801282    PPARG                         missense_variant   
8     rs1801282    PPARG                         missense_variant   
33    rs1801214     WFS1                         missense_variant   
36    rs1801214     WFS1                         missense_variant   
39    rs1801214     WFS1                         missense_variant   

          P_VALUE  
134  2.200000e-11  
136  2.200000e-11  
142  2.200000e-11  
2    5.000000e-09  
4    5.000000e-09  
6    5.000000e-09  
8    5.000000e-09  
33   1.300000e-08  
36   1.300000e-08  
39   1.300000e-08  


## Group by Genes

In [19]:
gene_counts = filtered_df['Gene'].value_counts()
print(gene_counts.head(10))


WFS1       7
PPARG      4
SLC30A8    3
Name: Gene, dtype: int64


## Export Filtered Set

In [20]:
filtered_df.to_csv(r"C:\Users\gargi\Desktop\Publication\Transcriptomic_and_Epigenomic_Features\high_impact_variants.csv", index=False)
